# About

Here we will:

- add metrics tracking
- add plots
- create experiments with different configurations
- compare metrics for different experiments using _DVC_

# Add metrics tracking

Edit `params.yaml`, stage `evaluate`:

1. add section `metrics`:
`params.yaml`
```yaml
...
evaluate:
    cmd: python src/stages/evaluate.py --config=params.yaml
    deps:
       ...
    params:
       ...
    metrics:
        
    outs:
    - reports/confusion_matrix.png
    - reports/metrics.json
    - reports/cm_plot.csv
```

2. move `reports/metrics.json` from `outs` to `metrics` section, you should get the following:

`params.yaml`
```yaml
...
evaluate:
    cmd: python src/stages/evaluate.py --config=params.yaml
    deps:
    - data/processed/test_iris.csv
    - models/model.joblib
    - src/stages/evaluate.py
    params:
    - base
    - data_split.testset_path
    - evaluate
    - featurize.target_column
    metrics:
    - reports/metrics.json
    outs:
    - reports/confusion_matrix.png
    - reports/cm_plot.csv
```


3. To add metrics file under git control turn off caching them:

`params.yaml`
```yaml
...
evaluate:
    cmd: python src/stages/evaluate.py --config=params.yaml
    deps:
    - data/processed/test_iris.csv
    - models/model.joblib
    - src/stages/evaluate.py
    params:
    - base
    - data_split.testset_path
    - evaluate
    - featurize.target_column
    metrics:
    - reports/metrics.json
    outs:
    - reports/confusion_matrix.png
    - reports/cm_plot.csv
```


# Add plots

Artifact `reports/cm_plot.csv` represents csv format of DVC plot metrics. DVC provides building plots by plot metrics files.

To make artifact plot metrics file move `reports/cm_plot.csv` from `outs` to `plots` section (like with metrics):


`params.yaml`
```yaml
...
evaluate:
    cmd: python src/stages/evaluate.py --config=params.yaml
    deps:
    - data/processed/test_iris.csv
    - models/model.joblib
    - src/stages/evaluate.py
    params:
    - base
    - data_split.testset_path
    - evaluate
    - featurize.target_column
    metrics:
    - reports/metrics.json:
        cache: false
    plots:
    - reports/cm_plot.csv:
        cache: false
    outs:
    - reports/confusion_matrix.png
```

# Experiment 1 - Tune LogisticRegression

## Create and run exeperiment


Create experiment and change parameter - add options for **C** hyperparamter in section `train:estimators:logreg:param_grid`:


```bash
dvc exp run -n exp1-tune-logreg \
    --set-param train.estimators.logreg.param_grid.C=[0.1,1.0,10]
```



as result you should have LogisticRegression config:

```yaml
...
train:
  cv: 3
  estimator_name: logreg

  estimators:

    logreg: # sklearn.linear_model.LogisticRegression
      param_grid: # params of GridSearchCV constructor
        C:
        - 0.1
        - 1.0
        - 10
        max_iter: [100]
        solver: [lbfgs]
        multi_class: [multinomial]
...

```


## Commit experiment results

```bash
git add .
git commit -m "Experiment 1 with LogisticRegression hyperparameters"
git tag -a "exp1_tune_logreg" -m "Experiment 1 with LogisticRegression hyperparameters"
dvc push
```

## Show metrics 

```bash
dvc metrics show
```

# Experiment 2 - Use SVM

## Create and run experiment

in section `train` change:
    - `estimator_name`
    - change `estimators:svm:param_grid:C`


```bash
dvc exp run -n exp2-svm \
    --set-param train.estimator_name=svm \
    --set-param train.estimators.svm.param_grid.C=[0.1,1.0,10]
```


as result you should have SVC config:

```yaml
...
train:
  cv: 3
  estimator_name: svm
  estimators:
        
    svm: # sklearn.svm.SVC
      param_grid:
        C:
        - 0.1
        - 1.0
        - 10
        kernel: [rbf, linear]
        gamma: [scale]
        degree: [3, 5]
...
```


## Commit experiment results

```bash
git add .
git commit -m "Experiment 2 with SVM estimator"
git tag -a "exp2_svm" -m "Experiment 2 with SVM estimator"
dvc push
```

## Show metrics 

```bash
dvc metrics show
```

# Compare experiments

1. List experiments:

```bash
dvc exp list --all
```
```
# output
940fee6:                                                              
        exp2-svm
step-6:
        exp1-tune-logreg

```

2. Compare current experiments`:

```bash
dvc exp diff --old  exp1-tune-logreg exp2-svm
```

# Push changes to remote repository

```bash
git push origin step-6
```

# DVC Studio


## View creation

1. Open [DVC Studio](https://studio.iterative.ai/) and sing up/sing in.

2. Create **View**: click **Add a view**

3. Select repository

## Work with View

1. Enter `View`

2. Navigate the branch `step-6`

3. Select one of commits and click `Show plots` - you'll see plots built by `DVC Studio` for selected commit

4. Select any **two commits** to compare it and click `Compare` - `DVS Studio` will build comparison table